# Part 01. Privacy Preserving Deep Learning

In [0]:
!pip install tf-encrypted

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b dev --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

remote: Enumerating objects: 620, done.
remote: Counting objects: 100% (620/620), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 1543 (delta 609), reused 607 (delta 607), pack-reused 923
Receiving objects: 100% (1543/1543), 1.80 MiB | 27.96 MiB/s, done.
Resolving deltas: 100% (1118/1118), completed with 234 local objects.
From https://github.com/openmined/PySyft
 * branch              HEAD       -> FETCH_HEAD

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@bc0f2c65a324.(none)')
zip_safe flag not set; analyzing archive contents...
  Using cached https://files.pythonhosted.org/packages/e7/81/011fef8766fb0ef681037ad6fee96168ee03a864464986cbaa23e5357704/lz4-3.0.2-cp36-cp36m-manylinux2010_x86_64.whl
ERROR: syft 0.2.0a2 has requireme

Processing /root/.cache/pip/wheels/35/f7/5c/9e8243838269ea93f05295708519a6e183fa6b515d9ce3b636/websocket-0.2.1-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f2/ca/5b5962361ed832847b6b2f9a2d0452c8c2f29a93baef850bb8ad067c7bf9/gevent-1.4.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/bf/45/142141aa47e01a5779f0fa5a53b81f8379ce8f2b1cd13df7d2f1d751ae42/greenlet-0.4.15-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: greenlet 0.4.15
    Uninstalling greenlet-0.4.15:
      Successfully uninstalled greenlet-0.4.15
  Found existing installation: gevent 1.4.0
    Uninstalling gevent-1.4.0:
      Successfully uninstalled gevent-1.4.0
  Found existing installation: websocket 0.2.1
    Uninstalling websocket-0.2.1:
      Successfully uninstalled websocket-0.2.1


  Using cached https://files.pythonhosted.org/packages/bb/d9/856af84843912e2853b1b6e898ac8b802989fcf9ecf8e8445a1da263bf3b/websockets-8.1-cp36-cp36m-manylinux2010_x86_64.whl
ERROR: syft 0.2.0a2 has requirement msgpack>=0.6.1, but you'll have msgpack 0.5.6 which is incompatible.
ERROR: syft 0.2.0a2 has requirement torch==1.3, but you'll have torch 1.3.1 which is incompatible.
ERROR: syft 0.2.0a2 has requirement torchvision==0.4.1, but you'll have torchvision 0.4.2 which is incompatible.
  Found existing installation: websockets 8.1
    Uninstalling websockets-8.1:
      Successfully uninstalled websockets-8.1


Processing /root/.cache/pip/wheels/ce/0d/5a/3c2ccf5522d1cd3ecdeb2af8c9889179d7a204376b1d5c042d/zstd-1.4.4.0-cp36-cp36m-linux_x86_64.whl
ERROR: syft 0.2.0a2 has requirement msgpack>=0.6.1, but you'll have msgpack 0.5.6 which is incompatible.
ERROR: syft 0.2.0a2 has requirement torch==1.3, but you'll have torch 1.3.1 which is incompatible.
ERROR: syft 0.2.0a2 has requirement torchvision==0.4.1, but you'll have torchvision 0.4.2 which is incompatible.
  Found existing installation: zstd 1.4.4.0
    Uninstalling zstd-1.4.4.0:
      Successfully uninstalled zstd-1.4.4.0


In [0]:
# Run this cell to see if things work
import sys

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

import syft as sy
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [0]:
x = torch.tensor([1, 2, 3, 4, 5])
y = x + x
print(y)

tensor([ 2,  4,  6,  8, 10])


## Section 1.1 - Sending Tensors to Bob's Machine

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [0]:
x = torch.tensor([1, 2, 3, 4, 5])
y = torch.tensor([1, 1, 1, 1, 1])

In [0]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

In [0]:
x_ptr

(Wrapper)>[PointerTensor | me:49573016498 -> bob:12310017438]

In [0]:
bob._objects

{12310017438: tensor([1, 2, 3, 4, 5]), 21459626547: tensor([1, 1, 1, 1, 1])}

In [0]:
x_ptr.location

<VirtualWorker id:bob #objects:2>

In [0]:
bob

<VirtualWorker id:bob #objects:2>

In [0]:
bob == x_ptr.location

True

In [0]:
x_ptr.id_at_location

12310017438

In [0]:
x_ptr.owner

<VirtualWorker id:me #objects:0>

In [0]:
me = sy.local_worker
me

<VirtualWorker id:me #objects:0>

In [0]:
me == x_ptr.owner

True

In [0]:
x_ptr

(Wrapper)>[PointerTensor | me:49573016498 -> bob:12310017438]

In [0]:
x_ptr.get()

tensor([1, 2, 3, 4, 5])

In [0]:
y_ptr

(Wrapper)>[PointerTensor | me:44808674936 -> bob:21459626547]

In [0]:
y_ptr.get()

tensor([1, 1, 1, 1, 1])

In [0]:
bob._objects

{}

## Section 1.2 - Using Tensor Pointers

In [0]:
x = torch.tensor([1, 2, 3, 4, 5]).send(bob)
y = torch.tensor([1, 1, 1, 1, 1]).send(bob)

In [0]:
z = x + y

In [0]:
z

(Wrapper)>[PointerTensor | me:6245857154 -> bob:87639727288]

In [0]:
z.get()

tensor([2, 3, 4, 5, 6])

In [0]:
x

(Wrapper)>[PointerTensor | me:88032362196 -> bob:33747754888]

In [0]:
y

(Wrapper)>[PointerTensor | me:43586111095 -> bob:51481539833]

In [0]:
z = torch.add(x, y)
z

(Wrapper)>[PointerTensor | me:46635722207 -> bob:68516958475]

In [0]:
z.get()

tensor([2, 3, 4, 5, 6])

In [0]:
x = torch.tensor([1, 2, 3, 4, 5.], requires_grad=True).send(bob)
y = torch.tensor([1, 1, 1, 1, 1.], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [0]:
z.backward()

(Wrapper)>[PointerTensor | me:61273540400 -> bob:59337690527]

In [0]:
x = x.get()

In [0]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [0]:
x.grad

tensor([1., 1., 1., 1., 1.])